In [1]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pyyaml ua-parser user-agents fake-useragent

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyshadow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from pyshadow.main import Shadow
import json
import time

In [6]:
json_file = 'riskware_output.json'

def update_json_file(file_path, new_data):
    with open(file_path, 'r') as file:
        existing_data = json.load(file)
    existing_data.update(new_data)
    with open(file_path, 'w') as file:
        json.dump(existing_data, file, indent=4)


In [7]:
global driver

In [8]:
captcha = False

In [9]:
def crawl(hash):
    global captcha
    url = f"https://www.virustotal.com/gui/file/{hash}/behavior"
    
    options = Options()
    ua = UserAgent()
    user_agent = ua.random
    print(user_agent)

    options.add_argument(f'--user-agent={user_agent}')
    options.add_argument('--window-size=600,400')

    #options.add_argument("--headless")
    driver = webdriver.Chrome(options=options, service=ChromeService(ChromeDriverManager(driver_version='125.0.6422.142').install()))
    driver.get(url)
    
    
    # ################################

    # wait for the page to load
    time.sleep(30)
    
    # check if the page is a captcha page
    if "captcha" in driver.title.lower():
        captcha = True
        print ("Captcha found")
        return "Captcha found"
    
    shadow = Shadow(driver)
    
    try:
        vt_ui_tree = shadow.find_element("vt-ui-mitre-tree#mitre-tree>vt-ui-expandable>span[slot='content']>div.position-relative")
    except:
        return "no data found"
    tree = BeautifulSoup(vt_ui_tree.get_attribute("outerHTML"), "html.parser")
    
    # MAIN TREE
    elements = tree.select("vt-ui-expandable-detail")
    time.sleep(5)
    driver.close()
    
    output = {}
    doc_name = hash
    doc_data = {
        "mitre": {}
    }
    
    
    for element in elements:
        # MITRE NAME
        label_content = element.select_one("span[slot='label-content']")
        a_tag = label_content.select_one("h5 > div > a")
        if a_tag:
            inner_text = a_tag.get_text(strip=True)
            doc_data["mitre"][inner_text] = {}

        # MITRE CODE
        span_tag = label_content.select_one("h5 > div > span")
        if span_tag:
            direct_text = ''.join(span_tag.find_all(text=True, recursive=False)).strip()
            parts = direct_text.split('$')
            ta_code = parts[-1]
            doc_data["mitre"][inner_text]["tcode"] = ta_code

        # MITRE INNER DATA
        table = element.select_one("span[slot='content'] > table")
        if table:
            for row in table.find_all("tr"):
                # PROTOCOL
                row_data = {}
                a_tag = row.select_one("td > span > div > a")
                if a_tag:
                    row_data["tcode"] = a_tag.get_text(strip=True)
                
                # PROTOCOL CODE
                span_tag = row.select_one("td > span > div > span")
                if span_tag:
                    direct_text = ''.join(span_tag.find_all(text=True, recursive=False)).strip()
                    parts = direct_text.split('$')
                    ta_code = parts[-1]
                    row_data["tcode"] = ta_code
                
                # PROTOCOL METHOD
                p_tag = row.select_one("td > span > div > p")
                if p_tag:
                    row_data["method"] = p_tag.get_text(strip=True)
                if row_data:
                    doc_data["mitre"][inner_text].setdefault("subcategories", []).append(row_data)

    return doc_data


In [ ]:
#  read txt file and loop through lines
with open('riskware.txt') as f:
    hashes = f.read().splitlines()

for hash in hashes:
    print(f"Processing {hash}")
    ow = crawl(hash)
    print(ow)
    if captcha == True:
        print("Captcha detected, stopping.")
        break
    output = { str(hash) : ow }
    update_json_file(json_file, output)
    #  remove line with hash value after processing
    with open('riskware.txt', 'r') as f:
        lines = f.readlines()
    with open('riskware.txt', 'w') as f:
        for line in lines:
            if line.strip("\n") != hash:
                f.write(line)
    


Processing 929F288F5A60CA53EB4CBCDD5F9DEAC679C31FA148C68248E7F9C148816A5497
Mozilla/5.0 (iPhone; CPU iPhone OS 17_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1 RDDocuments/8.4.8.940
no data found
Processing 92B3BF3C43F36F20116FCE1E6223E0CE40094771BC807623EF36243EB9330BA0
Mozilla/5.0 (iPhone; CPU iPhone OS 17_0_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1
no data found
Processing 92CCC4E96C94F934092D80BF9A150B4967E9A5F180F9FD9FCD5B0FD97243E69C
Mozilla/5.0 (Linux; Android 10; K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Mobile Safari/537.36
no data found
Processing 92CCE63AF9B3F945148847D536FE25B3985A98FA2727B1BE03EACC39221DB2E8
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36
no data found
Processing 92D6EB0538C28F6B67BE1FFA341548F89B728E3300651860C08A66B475DD9EDB
Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv